### Import packages

In [1]:
import pandas as pd
import numpy
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

%matplotlib inline

/Users/jas/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/Users/jas/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 1. Read in relevant data (3 sources)

In [2]:
# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html
# try with read_excel
# hist_regional_fuel = pd.read_excel('pswrgvwall.xls', sheet_name = 12, header = 2)
# hist_regional_fuel.to_csv('hist_reg_gas.csv')

## Source 1: Scrape Historical WTI data
- source: https://www.eia.gov/opendata/qb.php?sdid=PET.RWTC.D
- 1. Function defined to pull from data source (wti_hist_source())
- 2. Function defined to read from local csv (wti_hist_local())

In [3]:
# wti_url = 'https://www.eia.gov/opendata/qb.php?sdid=PET.RWTC.D'

def wti_hist_source(url):
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'lxml')

    # scrape table head
    wti_col = soup.thead.find_all('th')
    wti_columns = [x.get_text() for x in wti_col]

    # scrape table rows
    wti_r = soup.tbody.find_all('tr')
    wti_rows = []
    for x in wti_r:
        td_rset = x.find_all('td')
        wti_ro = [x.text for x in td_rset]
        wti_rows.append(wti_ro)

In [4]:
def wti_cleansave_df(dict_name, df_name): # wti_dict, wti_df are appropriate
    dict_name = {}
    dict_name['data_desc'] = [x[0] for x in wti_rows]
    dict_name['date'] = [x[1] for x in wti_rows]
    dict_name['freq'] = [x[2] for x in wti_rows]
    dict_name['price'] = [x[3] for x in wti_rows]
    dict_name['unit'] = [x[4] for x in wti_rows]

    # convert to dataframe for better maneagability (future merging and analyzing)
    df_name = pd.DataFrame(wti_dict)

    # define data types
    df_name['data_desc'] = df_name['data_desc'].astype('category')
    df_name['date'] = pd.to_datetime(df_name['date'])
    df_name['freq'] = df_name['freq'].astype('category')
    df_name['price'] = pd.to_numeric(df_name['price'])
    df_name['unit'] = df_name['unit'].astype('category')
    
    
    # save updated wti_spot pricing info
    df_name.to_pickle(str(df_name)+'_current.pkl')
    
    return(df_name.info())

In [5]:
# read in WTI - Daily Crude prices
# def wti_only_price_pkl(df_name):
#     df_name = pd.read_pickle(str(df_name)+'_current.pkl')
#     df_name['date'] = pd.to_datetime(wti_daily_df['date'])
#     df_name.set_index('date', inplace=True)
#     df_name.drop(['Unnamed: 0','freq','data_desc','unit'] ,axis=1 ,inplace=True)
#     df_name.rename(columns={'price':'CUSHOK_WTI_spot_DPB'}, inplace=True)
#     return(df_name.head(3))

In [6]:
# def wti_only_price_csv(df_name):
#     df_name = pd.read_csv('wti_df_16apr18.csv')
#     df_name['date'] = pd.to_datetime(df_name['date'])
#     df_name.set_index('date', inplace=True)
#     df_name.drop(['Unnamed: 0','freq','data_desc','unit'] ,axis=1 ,inplace=True)
#     df_name.rename(columns={'price':'CUSHOK_WTI_spot_DPB'}, inplace=True)
#     return(df_name.head(3))

In [7]:
# wti_only_price_csv(wti_daily_df)

In [8]:
wti_crudespot_df = pd.read_csv('Gas_Data/wti_df_16apr18.csv')
wti_crudespot_df['date'] = pd.to_datetime(wti_crudespot_df['date'])
wti_crudespot_df.set_index('date', inplace=True)
wti_crudespot_df.drop(['Unnamed: 0','freq','data_desc','unit'] ,axis=1 ,inplace=True)
wti_crudespot_df.rename(columns={'price':'CUSHOK_WTI_spot_DPB'}, inplace=True)
wti_crudespot_df.head(3)

,CUSHOK_WTI_spot_DPB
date,
2018-04-09,63.40
2018-04-06,62.03
2018-04-05,63.53


## Source 2: BP Energy Report
(bp-statistical-review-of-world-energy-2017-underpinning-data.xlsx)
TBD = Thousand Barrels Daily
BB = Billion Barrels
MTOE = Million Tonne Oil Equivalent

1. Oil Production - Barrels
2. Oil - Refinery capacities
3. Oil - Proved Reserves History
4. Oil - Trade movements
5. Oil - Refiner throughput
6. Renewables/Coal/Nuclear... consider later

In [9]:
def set_bp_year_dtindex(df_name):
    df_name.reset_index(inplace=True)
    df_name.rename(columns = {'index':'year'}, inplace=True)
    df_name['year'] = pd.to_datetime(df_name['year'], format='%Y', errors='coerce')
    df_name.set_index('year', inplace=True)
    
def set_eia_week_dtindex(df_name):
    df_name.reset_index(inplace=True)
    df_name.rename(columns = {'Date':'week'}, inplace=True)
    df_name['week'] = pd.to_datetime(df_name['week'], format='%Y-%m-%d', errors='coerce')
    df_name.set_index('week', inplace=True)
    
def set_eia_month_dtindex(df_name):
    df_name.reset_index(inplace=True)
    df_name.rename(columns = {'Date':'month'}, inplace=True)
    df_name['month'] = pd.to_datetime(df_name['month'], format='%Y-%m', errors='coerce')
    df_name.set_index('month', inplace=True)

In [10]:
# Read in BP data from xlsx
# Oil Production - Barrels; US, Total World; TBD
#bp_production_df = bp_production_df[bp_production_df.index.str.startswith('Total') == False]
bp_production_df = pd.read_excel('Gas_Data/bp-statistical-review-of-world-energy-2017-underpinning-data.xlsx', header = 2, index_col = 0, sheet_name = 'Oil Production - Barrels')
bp_production_df = bp_production_df.loc[['US', 'Total World']]
bp_production_df = bp_production_df.transpose()
bp_production_df.rename(columns = {'US':'US_production_TBD', 'Total World':'Global_production_TBD'}, inplace=True)
set_bp_year_dtindex(bp_production_df)

In [11]:
# Read in BP data from xlsx
# Oil - Refinery capacities; US, Total World; TBD
bp_refcapac_df = pd.read_excel('Gas_Data/bp-statistical-review-of-world-energy-2017-underpinning-data.xlsx', header = 2, index_col = 0, sheet_name = 'Oil - Refinery capacities')
bp_refcapac_df = bp_refcapac_df.loc[['US', 'Total World']]
bp_refcapac_df = bp_refcapac_df.transpose()
bp_refcapac_df.rename(columns = {'US':'US_refcapac_TBD', 'Total World':'Global_refcapac_TBD'}, inplace=True)
set_bp_year_dtindex(bp_refcapac_df)

In [12]:
# Read in BP data from xlsx
# Oil - Proved reserves history; US, Total World; BB
bp_reserves_df = pd.read_excel('Gas_Data/bp-statistical-review-of-world-energy-2017-underpinning-data.xlsx', header = 2, index_col = 0, sheet_name = 'Oil - Proved reserves history')
bp_reserves_df = bp_reserves_df.loc[['US', 'Total World']]
bp_reserves_df = bp_reserves_df.transpose()
bp_reserves_df.rename(columns = {'US':'US_PReserves_BB', 'Total World':'Global_PReserves_BB'}, inplace=True)
set_bp_year_dtindex(bp_reserves_df)

In [13]:
# Read in BP data from xlsx
# Oil - Trade movements; US, Total World; TBD
bp_trade_df = pd.read_excel('Gas_Data/bp-statistical-review-of-world-energy-2017-underpinning-data.xlsx', header = 2, index_col = 0, sheet_name = 'Oil - Trade movements')
bp_trade_df = bp_trade_df.loc['US'] # other option: iloc[[5, 20]
bp_trade_df = bp_trade_df.transpose()
# below must do differently to account for two rows with same index name
bp_trade_df.columns.values[1] = 'US_Exports_TBD'
bp_trade_df.columns.values[0] = 'US_Imports_TBD'
set_bp_year_dtindex(bp_trade_df)

In [14]:
# Read in BP data from xlsx
# Primary Energy Consumption; US, Total World; MTOE
bp_consumption_df = pd.read_excel('Gas_Data/bp-statistical-review-of-world-energy-2017-underpinning-data.xlsx', header = 2, index_col = 0, sheet_name = 'Primary Energy Consumption')
bp_consumption_df = bp_consumption_df.loc[['US', 'Total World']]
bp_consumption_df = bp_consumption_df.transpose()
bp_consumption_df = bp_consumption_df.rename(columns={'US':'US_consumption_MTOE', 'Total World':'Global_consumption_MTOE'})
set_bp_year_dtindex(bp_consumption_df)

In [15]:
# Read in BP data from xlsx
# Oil - Refinery throughput; US, Total World; TBD
bp_refthru_df = pd.read_excel('Gas_Data/bp-statistical-review-of-world-energy-2017-underpinning-data.xlsx', header = 2, index_col = 0, sheet_name = 'Oil - Refinery throughput')
bp_refthru_df = bp_refthru_df.loc[['US', 'Total World']]
bp_refthru_df = bp_refthru_df.transpose()
bp_refthru_df = bp_refthru_df.rename(columns={'US':'US_refthru_TBD', 'Total World':'Global_refthru_TBD'})
set_bp_year_dtindex(bp_refthru_df)

## Source 3: EIA Weekly Report

In [16]:
# Read in EAI data from xls
# Daily data
wti_crudefuturesc1_df = pd.read_excel('Gas_Data/wti_crudefuturesc1_daily.xls', sheet_name='Data 1', index_col=0, header=2)
set_eia_week_dtindex(wti_crudefuturesc1_df)
wti_crudefuturesc3_df = pd.read_excel('Gas_Data/wti_crudefuturesc3_daily.xls', sheet_name='Data 1', index_col=0, header=2)
set_eia_week_dtindex(wti_crudefuturesc3_df)

# below HU/RBOBc1 & c3 to be appended on Jun 2016
nyh_HUfuturesc1_df = pd.read_excel('Gas_Data/nyharbor_HUfuturesc1_daily.xls', sheet_name='Data 1', index_col=0, header=2)
set_eia_week_dtindex(nyh_HUfuturesc1_df)
nyh_RBOBfuturesc1_df = pd.read_excel('Gas_Data/nyharbor_RBOBfuturesc1_daily.xls', sheet_name='Data 1', index_col=0, header=2)
set_eia_week_dtindex(nyh_RBOBfuturesc1_df)
nyh_HUfuturesc3_df = pd.read_excel('Gas_Data/nyharbor_HUfuturesc3_daily.xls', sheet_name='Data 1', index_col=0, header=2)
set_eia_week_dtindex(nyh_HUfuturesc3_df)
nyh_RBOBfuturesc3_df = pd.read_excel('Gas_Data/nyharbor_RBOBfuturesc3_daily.xls', sheet_name='Data 1', index_col=0, header=2)
set_eia_week_dtindex(nyh_RBOBfuturesc3_df)

# Weekly data
wa_retail_df = pd.read_excel('Gas_Data/wash_retailgas_week.xls', sheet_name='Data 1', index_col=0, header=2)
set_eia_week_dtindex(wa_retail_df)
us_netcrudeinputs_df = pd.read_excel('Gas_Data/us_netcrudeinputs_week.xls', sheet_name='Data 1', index_col=0, header=2)
set_eia_week_dtindex(us_netcrudeinputs_df)
us_retail_df = pd.read_excel('Gas_Data/us_retailregprice_week.xls', sheet_name='Data 1', index_col=0, header=2)
us_retail_df_slim = us_retail_df[['Weekly U.S. Regular Conventional Retail Gasoline Prices  (Dollars per Gallon)']]
set_eia_week_dtindex(us_retail_df_slim)
us_storage_df = pd.read_excel('Gas_Data/us_storage_week.xls', sheet_name='Data 1', index_col=0, header=2)
set_eia_week_dtindex(us_storage_df)

# Monthly data
us_oilproduction_df = pd.read_excel('Gas_Data/us_oilproduction_month.xls', sheet_name='Data 1', index_col=0, header=2)
set_eia_month_dtindex(us_oilproduction_df)
us_pctyieldmgas_df = pd.read_excel('Gas_Data/us_refinyieldmgas_week.xls', sheet_name='Data 1', index_col=0, header=2)
set_eia_week_dtindex(us_pctyieldmgas_df)

/Users/jas/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Users/jas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [17]:
nyh_RBOBfuturesc1_df.columns

Index(['New York Harbor Reformulated RBOB Regular Gasoline Future Contract 1 (Dollars per Gallon)'], dtype='object')

In [18]:
# concat or append reggasfuturesc1 & c3 to rbobc1 & c3
# Prep contract 1 data
nyh_HUfuturesc1_df = nyh_HUfuturesc1_df.loc[nyh_HUfuturesc1_df.index.year < 2006]
nyh_RBOBfuturesc1_df = nyh_RBOBfuturesc1_df.loc[nyh_RBOBfuturesc1_df.index.year > 2005]
nyh_HUfuturesc1_df.rename(columns={'New York Harbor Regular Gasoline Future Contract 1 (Dollars per Gallon)': 'nyh_HU_RBOBc1_DPG'}, inplace=True)
nyh_RBOBfuturesc1_df.rename(columns={'New York Harbor Reformulated RBOB Regular Gasoline Future Contract 1 (Dollars per Gallon)':'nyh_HU_RBOBc1_DPG'}, inplace=True)

# Merge contract 1 data
framesc1 = [nyh_HUfuturesc1_df, nyh_RBOBfuturesc1_df]
nyh_HU_RBOB_futuresc1_df = pd.concat(framesc1)

# Prep contract 3 data
nyh_HUfuturesc3_df = nyh_HUfuturesc3_df.loc[nyh_HUfuturesc1_df.index.year < 2006]
nyh_RBOBfuturesc3_df = nyh_RBOBfuturesc3_df.loc[nyh_RBOBfuturesc1_df.index.year > 2005]
nyh_HUfuturesc3_df.rename(columns={'New York Harbor Regular Gasoline Future Contract 3 (Dollars per Gallon)': 'nyh_HU_RBOBc3_DPG'}, inplace=True)
nyh_RBOBfuturesc3_df.rename(columns={'New York Harbor Reformulated RBOB Regular Gasoline Future Contract 3 (Dollars per Gallon)':'nyh_HU_RBOBc3_DPG'}, inplace=True)
# Merge contract 3 data
framesc3 = [nyh_HUfuturesc3_df, nyh_RBOBfuturesc3_df]
nyh_HU_RBOB_futuresc3_df = pd.concat(framesc3)

# 2. Consolidate data

In [19]:
# could build a function to load all of these dfs... later.
df_joined = wa_retail_df.join([wti_crudefuturesc1_df, wti_crudefuturesc3_df, wti_crudespot_df, nyh_HU_RBOB_futuresc1_df, nyh_HU_RBOB_futuresc3_df, us_storage_df, us_retail_df_slim, us_pctyieldmgas_df, us_netcrudeinputs_df, us_oilproduction_df, bp_production_df, bp_refcapac_df, bp_reserves_df, bp_trade_df, bp_consumption_df, bp_refthru_df], how='outer')

#df_joined['U.S. Refinery Yield of Finished Motor Gasoline (Percent)'].fillna(method='bfill', inplace=True)
df_joined.fillna(method='ffill', inplace=True)
#tester = df_joined[['Weekly Washington Regular All Formulations Retail Gasoline Prices  (Dollars per Gallon)','U.S. Refinery Yield of Finished Motor Gasoline (Percent)']]
df_joined.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10776 entries, 1920-01-15 to NaT
Data columns (total 23 columns):
Weekly Washington Regular All Formulations Retail Gasoline Prices  (Dollars per Gallon)    4693 non-null float64
Cushing, OK Crude Oil Future Contract 1 (Dollars per Barrel)                               9967 non-null float64
Cushing, OK Crude Oil Future Contract 3 (Dollars per Barrel)                               9970 non-null float64
CUSHOK_WTI_spot_DPB                                                                        9259 non-null float64
nyh_HU_RBOBc1_DPG                                                                          9516 non-null float64
nyh_HU_RBOBc3_DPG                                                                          9536 non-null float64
Weekly U.S. Ending Stocks of Crude Oil and Petroleum Products  (Thousand Barrels)          8210 non-null float64
Weekly U.S. Regular Conventional Retail Gasoline Prices  (Dollars per Gallon)             

In [20]:
df_joined.rename(columns={'Weekly Washington Regular All Formulations Retail Gasoline Prices  (Dollars per Gallon)':'wa_retail_wk_DPG', 
                          'Cushing, OK Crude Oil Future Contract 1 (Dollars per Barrel)':'wti_crudefuturesc1_DPB', 
                          'CUSHOK_WTI_spot_DPB': 'wti_OKcrudespot_DPB',
                          'Cushing, OK Crude Oil Future Contract 1 (Dollars per Barrel)':'wti_crudefuturesc1_DPG',
                          'Cushing, OK Crude Oil Future Contract 3 (Dollars per Barrel)':'wti_crudefuturesc3_DPG',
                          'New York Harbor Reformulated RBOB Regular Gasoline Future Contract 1 (Dollars per Gallon)': 'wti_crudefuturesc3_DPG',
                    'Weekly U.S. Ending Stocks of Crude Oil and Petroleum Products  (Thousand Barrels)':'us_crudestock_TB', 
                   'Weekly U.S. Regular Conventional Retail Gasoline Prices  (Dollars per Gallon)':'us_retail_wk_DPG',
                   'U.S. Refinery Yield of Finished Motor Gasoline (Percent)':'us_refyield_wk_pct',
                   'Weekly U.S. Refiner Net Input of Crude Oil  (Thousand Barrels per Day)':'us_refcrudeinput_wk_TBD', 
                   'U.S. Field Production of Crude Oil (Thousand Barrels per Day)':'us_production_wk_TBD'
                  }, inplace=True)

In [21]:
# add (1) wti_crudefutures, (2) wti_crudespot (change from 'wti_df'), and (3) nyharbor_RBOB

In [22]:
df_joined = df_joined[['us_retail_wk_DPG', 'wti_crudefuturesc1_DPG', 'wti_crudefuturesc3_DPG', 'nyh_HU_RBOBc1_DPG', 
                       'nyh_HU_RBOBc3_DPG', 'wti_OKcrudespot_DPB', 'us_crudestock_TB','us_refyield_wk_pct',
       'us_refcrudeinput_wk_TBD', 'us_production_wk_TBD', 'US_production_TBD',
       'Global_production_TBD', 'US_refcapac_TBD', 'Global_refcapac_TBD',
       'US_PReserves_BB', 'Global_PReserves_BB', 'US_Imports_TBD',
       'US_Exports_TBD', 'US_consumption_MTOE', 'Global_consumption_MTOE',
       'US_refthru_TBD', 'Global_refthru_TBD']]

In [23]:
# first get average US gas prices, then WA can be a 'secondary forecast'
# df.drop('wa_retail_wk_DPG', axis=1, inplace=True)

In [24]:
# drop null datetimes ('NaT') with: 
df_joined = df_joined[df_joined.index.notnull()]
# drop null values
df_joined.dropna(inplace=True)
# limit df results to focal years
# df = df.loc['1990':'2016']

# Pickle That!

In [25]:
df_joined.to_pickle('Gas_Data/raw_joined_df.pkl')

In [26]:
# df = pd.read_pickle('raw_joined_df.pkl')

In [27]:
# df.head()

In [28]:
df_joined.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6684 entries, 1993-01-15 to 2018-04-24
Data columns (total 22 columns):
us_retail_wk_DPG           6684 non-null float64
wti_crudefuturesc1_DPG     6684 non-null float64
wti_crudefuturesc3_DPG     6684 non-null float64
nyh_HU_RBOBc1_DPG          6684 non-null float64
nyh_HU_RBOBc3_DPG          6684 non-null float64
wti_OKcrudespot_DPB        6684 non-null float64
us_crudestock_TB           6684 non-null float64
us_refyield_wk_pct         6684 non-null float64
us_refcrudeinput_wk_TBD    6684 non-null float64
us_production_wk_TBD       6684 non-null float64
US_production_TBD          6684 non-null float64
Global_production_TBD      6684 non-null float64
US_refcapac_TBD            6684 non-null float64
Global_refcapac_TBD        6684 non-null float64
US_PReserves_BB            6684 non-null float64
Global_PReserves_BB        6684 non-null float64
US_Imports_TBD             6684 non-null float64
US_Exports_TBD             6684 non-null f